## Demo: WhileActivity with ShiftAmountActivity
The basic steps to set up an OpenCLSim simulation are:
* Import libraries
* Initialise simpy environment
* Define object classes
* Create objects
  * Create sites
  * Create vessels
  * Create activities
* Register processes and run simpy

----

The combination of a while activity with a shift amount activity can be used to represent the loading or unloading of a vessel, where coordination of the process is represented on the granularity of the amount shifted in one iteration. In this particular example we define the time it takes to shift an amount of 1, and define a while activity that executes this activity until the vessel container is full.

#### 0. Import libraries

In [1]:
import datetime, time
import simpy

import shapely.geometry
import pandas as pd

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

#### 1. Initialise simpy environment

In [2]:
# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)

#### 2. Define object classes

In [3]:
# create a Site object based on desired mixin classes
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

# create a TransportProcessingResource object based on desired mixin classes
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)

#### 3. Create objects
##### 3.1. Create site object(s)

In [4]:
# prepare input data for from_site
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)
data_from_site = {"env": my_env,
                  "name": "from_site",
                  "geometry": location_from_site,
                  "capacity": 10,
                  "level": 10
                 }
# instantiate from_site 
from_site = Site(**data_from_site)

# prepare input data for to_site
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)
data_to_site = {"env": my_env,
                "name": "to_site",
                "geometry": location_to_site,
                "capacity": 10,
                "level": 0
               }
# instantiate to_site 
to_site = Site(**data_to_site)

##### 3.2. Create vessel object(s)

In [5]:
# prepare input data for vessel_01
data_vessel01 = {"env": my_env,
                 "name": "vessel01",
                 "geometry": location_from_site, 
                 "loading_rate": 1,
                 "unloading_rate": 1,
                 "capacity": 5,
                 "compute_v": lambda x: 10
               }
# instantiate vessel_01 
vessel01 = TransportProcessingResource(**data_vessel01)

##### 3.3 Create activity/activities

In [6]:
# initialise registry
registry = {}

In [7]:
# create a list of the sub processes
sub_processes = [
    model.ShiftAmountActivity(
        env=my_env,
        name="Shift 1 amount of payload",
        registry=registry,
        processor=vessel01,
        origin=from_site,
        destination=vessel01,
        amount=1,
        duration=20,
    )
]

# create a 'while activity' that is made up of the 'sub_processes'
while_activity = model.WhileActivity(
    env=my_env,
    name="while_activity",
    registry=registry,
    sub_processes=sub_processes,
    condition_event=[{"type": "container", "concept": vessel01, "state": "full"}],
)

We execute the sub process (shift amount) until the container of vessel01 is full.

#### 4. Register processes and run simpy

In [8]:
# initate the simpy processes defined in the 'while activity' and run simpy
model.register_processes([while_activity])
my_env.run()

#### 5. Inspect results
##### 5.1 Inspect logs
The container of vessel01 has a capacity of 5. The shift amount activity shifts 1 amount of payload in 20 time untis. We can see from the log that the shift amount activity was executed 5 times. The simulation ends at 100 time units, when the vessel01 container level is 5. 

In [9]:
display(plot.get_log_dataframe(vessel01, [while_activity, *sub_processes]))

,Activity,Timestamp,ActivityState,geometry,container level
0,Shift 1 amount of payload,1970-01-01 00:00:00,START,POINT (4.18055556 52.18664444),0.0
1,Shift 1 amount of payload,1970-01-01 00:00:20,STOP,POINT (4.18055556 52.18664444),1.0
2,Shift 1 amount of payload,1970-01-01 00:00:20,START,POINT (4.18055556 52.18664444),1.0
3,Shift 1 amount of payload,1970-01-01 00:00:40,STOP,POINT (4.18055556 52.18664444),2.0
4,Shift 1 amount of payload,1970-01-01 00:00:40,START,POINT (4.18055556 52.18664444),2.0
5,Shift 1 amount of payload,1970-01-01 00:01:00,STOP,POINT (4.18055556 52.18664444),3.0
6,Shift 1 amount of payload,1970-01-01 00:01:00,START,POINT (4.18055556 52.18664444),3.0
7,Shift 1 amount of payload,1970-01-01 00:01:20,STOP,POINT (4.18055556 52.18664444),4.0
8,Shift 1 amount of payload,1970-01-01 00:01:20,START,POINT (4.18055556 52.18664444),4.0
9,Shift 1 amount of payload,1970-01-01 00:01:40,STOP,POINT (4.18055556 52.18664444),5.0


In [10]:
display(plot.get_log_dataframe(while_activity, [while_activity, *sub_processes]))

,Activity,Timestamp,ActivityState,type,ref
0,while_activity,1970-01-01 00:00:00,START,NaN,NaN
1,while_activity,1970-01-01 00:00:00,START,subprocess,f3d569c9-ae8a-46d9-b130-f28652a8d583
2,while_activity,1970-01-01 00:00:20,STOP,subprocess,f3d569c9-ae8a-46d9-b130-f28652a8d583
3,while_activity,1970-01-01 00:00:20,START,subprocess,f3d569c9-ae8a-46d9-b130-f28652a8d583
4,while_activity,1970-01-01 00:00:40,STOP,subprocess,f3d569c9-ae8a-46d9-b130-f28652a8d583
5,while_activity,1970-01-01 00:00:40,START,subprocess,f3d569c9-ae8a-46d9-b130-f28652a8d583
6,while_activity,1970-01-01 00:01:00,STOP,subprocess,f3d569c9-ae8a-46d9-b130-f28652a8d583
7,while_activity,1970-01-01 00:01:00,START,subprocess,f3d569c9-ae8a-46d9-b130-f28652a8d583
8,while_activity,1970-01-01 00:01:20,STOP,subprocess,f3d569c9-ae8a-46d9-b130-f28652a8d583
9,while_activity,1970-01-01 00:01:20,START,subprocess,f3d569c9-ae8a-46d9-b130-f28652a8d583


##### 5.2 Visualise gantt charts

In [11]:
plot.get_gantt_chart([while_activity, *sub_processes, vessel01])

#### Sandbox

We can inspect the resulting levels of the container objects in vessel01 and from_site as follows. 

In [12]:
vessel01.container.get_level()

5.0

In [13]:
from_site.container.get_level()

5